In [27]:
import pandas as pd
import numpy as np
import time
import faiss
from memory_profiler import memory_usage
from sklearn.neighbors import NearestNeighbors
from imblearn.over_sampling import SMOTE
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler

In [28]:
IMBALANCED_DATA_PATH = '../data/01_raw/original_imbalanced.csv'
df_imbalanced = pd.read_csv(IMBALANCED_DATA_PATH)
df_minority = df_imbalanced[df_imbalanced['target'] == 1].drop('target', axis=1)
df_minority_numeric = df_minority.select_dtypes(include=np.number).astype('float32')


In [ ]:
def profile_smote(data):
    n_samples = len(data)
    
    n_majority = int(n_samples * 0.8)
    n_minority = n_samples - n_majority  
    
    y_synthetic = np.concatenate([
        np.zeros(n_majority), 
        np.ones(n_minority)
    ])
    
    np.random.seed(42)  
    np.random.shuffle(y_synthetic)
    
    smote = SMOTE(random_state=42)
    X_resampled, y_resampled = smote.fit_resample(data, y_synthetic)
    return X_resampled, y_resampled

def profile_graph_exact(data):
    nn = NearestNeighbors(n_neighbors=min(6, len(data)), algorithm='ball_tree')
    nn.fit(data)
    # Perform a query to simulate actual usage
    distances, indices = nn.kneighbors(data[:min(100, len(data))])
    return distances, indices

def profile_graph_optimized_batch(data):
    """Profile the optimized batch-based graph generation method."""
    # Scale data
    scaler = StandardScaler()
    data_scaled = scaler.fit_transform(data)
    
    # Build k-nearest neighbors graph
    n_neighbors = min(6, len(data))
    nn = NearestNeighbors(
        n_neighbors=n_neighbors,
        algorithm='ball_tree',
        n_jobs=-1
    )
    nn.fit(data_scaled)
    distances, indices = nn.kneighbors(data_scaled)
    
    n_samples_to_generate = min(1000, len(data) * 2) 
    batch_size = min(100, n_samples_to_generate)
    n_batches = (n_samples_to_generate + batch_size - 1) // batch_size
    
    all_synthetic = []
    
    for batch_idx in range(n_batches):
        current_batch_size = min(batch_size, n_samples_to_generate - batch_idx * batch_size)
        
        random_node_indices = np.random.randint(0, len(data), size=current_batch_size)
        
        batch_synthetic = np.zeros((current_batch_size, data.shape[1]))
        
        for i, node_idx in enumerate(random_node_indices):
            if len(indices[node_idx]) > 1:
                neighbor_idx = np.random.choice(indices[node_idx][1:])
                
                interpolation_ratio = np.random.rand()
                batch_synthetic[i] = (
                    data[node_idx] * interpolation_ratio + 
                    data[neighbor_idx] * (1 - interpolation_ratio)
                )
            else:
                batch_synthetic[i] = data[node_idx]  
        
        all_synthetic.append(batch_synthetic)
    
    return np.vstack(all_synthetic) if all_synthetic else data[:10]

def profile_graph_ann(data):
    d = data.shape[1]
    index = faiss.IndexFlatL2(d)
    data_float32 = data.astype('float32') if data.dtype != np.float32 else data
    index.add(data_float32)
    query_data = data_float32[:min(100, len(data_float32))]
    distances, indices = index.search(query_data, min(6, len(data_float32)))
    return distances, indices


In [ ]:
data_sizes = [1000, 5000, 10000, 20000, 40000, len(df_minority_numeric)]
results = []

methods = {
    "SMOTE (sklearn)": profile_smote,
    "Graph-Based (Exact NN)": profile_graph_exact,
    "Graph-Based (Optimized Batch)": profile_graph_optimized_batch,
    "Graph-Based (ANN w/ FAISS)": profile_graph_ann
}

for size in data_sizes:
    print(f"\nBenchmarking with {size} samples")
    actual_size = min(size, len(df_minority_numeric))
    subset = df_minority_numeric.sample(n=actual_size, random_state=42).to_numpy()

    for name, func in methods.items():
        print(f"Profiling {name}...")
        start_time = time.time()
        
        try:
            mem_use = memory_usage((func, (subset,)), interval=0.1, max_usage=True)
            
            end_time = time.time()
            time_taken = end_time - start_time
            
            results.append({
                "Method": name,
                "Data Size": actual_size,
                "Time (s)": round(time_taken, 4),
                "Peak Memory (MB)": round(mem_use, 2)
            })
            
        except Exception as e:
            print(f"Error with {name} at size {actual_size}: {str(e)}")
            results.append({
                "Method": name,
                "Data Size": actual_size,
                "Time (s)": None,
                "Peak Memory (MB)": None
            })

df_results = pd.DataFrame(results)
print("\n" + "="*50)
print("BENCHMARKING RESULTS")
print("="*50)
print(df_results.to_string(index=False))

print("\n" + "="*50)
print("PERFORMANCE ANALYSIS")
print("="*50)

for method in methods.keys():
    method_data = df_results[df_results['Method'] == method].copy()
    method_data = method_data.dropna()  
    
    if not method_data.empty:
        print(f"\n{method}:")
        print(f"  Time range: {method_data['Time (s)'].min():.4f}s - {method_data['Time (s)'].max():.4f}s")
        print(f"  Memory range: {method_data['Peak Memory (MB)'].min():.2f}MB - {method_data['Peak Memory (MB)'].max():.2f}MB")
        
        if len(method_data) > 1:
            time_scaling = method_data['Time (s)'].iloc[-1] / method_data['Time (s)'].iloc[0]
            memory_scaling = method_data['Peak Memory (MB)'].iloc[-1] / method_data['Peak Memory (MB)'].iloc[0]
            size_scaling = method_data['Data Size'].iloc[-1] / method_data['Data Size'].iloc[0]
            
            print(f"  Time scaling factor: {time_scaling:.2f}x for {size_scaling:.1f}x data increase")
            print(f"  Memory scaling factor: {memory_scaling:.2f}x for {size_scaling:.1f}x data increase")


Benchmarking with 1000 samples
Profiling SMOTE (sklearn)...
Profiling Graph-Based (Exact NN)...
Profiling Graph-Based (Optimized Batch)...
Profiling Graph-Based (ANN w/ FAISS)...

Benchmarking with 5000 samples
Profiling SMOTE (sklearn)...
Profiling Graph-Based (Exact NN)...
Profiling Graph-Based (Optimized Batch)...
Profiling Graph-Based (ANN w/ FAISS)...

Benchmarking with 10000 samples
Profiling SMOTE (sklearn)...
Profiling Graph-Based (Exact NN)...
Profiling Graph-Based (Optimized Batch)...
Profiling Graph-Based (ANN w/ FAISS)...

Benchmarking with 20000 samples
Profiling SMOTE (sklearn)...
Profiling Graph-Based (Exact NN)...
Profiling Graph-Based (Optimized Batch)...
Profiling Graph-Based (ANN w/ FAISS)...

Benchmarking with 40000 samples
Profiling SMOTE (sklearn)...
Profiling Graph-Based (Exact NN)...
Profiling Graph-Based (Optimized Batch)...
Profiling Graph-Based (ANN w/ FAISS)...

Benchmarking with 5564 samples
Profiling SMOTE (sklearn)...
Profiling Graph-Based (Exact NN)...
